In [8]:
from typing import Annotated, Sequence, TypedDict
from dotenv import load_dotenv  
from langchain_core.messages import BaseMessage # The foundational class for all message types in LangGraph
from langchain_core.messages import ToolMessage # Passes data back to LLM after it calls a tool such as the content and the tool_call_id
from langchain_core.messages import SystemMessage # Message for providing instructions to the LLM
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END, START
from langgraph.prebuilt import ToolNode
load_dotenv()

True

#### Pre-requisite : Meaning of different imports done above

In [9]:
""" Meaning of different imports done above """

# 1) Annotated: provides additional context without affecting the type itself
email = Annotated[str, "This has to follow abc@xyz.com format"]; # -> so this says email will be a string with additionl metadata attached as "This has to follow abc@xyz.com format"

print(email.__metadata__)

# 2) Sequence: To automatically handle the state updates for sequences such as by adding new messages to a chat history

'''
  3) Reducer function (add_messages in above imports): a rulde that controls how updates from nodes are combined with the existing state.
  
  Simply put: it tells us how to merge new data into the current state
'''

# Without Reducer
old_state = {"messages": ["HI!"]}
update_from_node = {"messages": ["Nice to meet you!"]}
new_state = {"messages": ["Nice to meet you!"]}

# With reducer
old_state = {"messages": ["HI!"]}
update_from_node = {"messages": ["Nice to meet you!"]}
new_state = {"messages": ["Hi!", "Nice to meet you!"]}

# Note: for simpler applications, we can use .append() method to append new messages to the "messages" list but it won't work when we actually introduce cmoplexity (like LLM with tools) because a reducer does more than just .append(), like correct format, adding ids and sometimes the framework (here, langgraph) expects you to just return the updates not directly mutate the state (compare this to react which also expects you to use the state update function and not modify the state directly)



('This has to follow abc@xyz.com format',)


#### Now, the ReAct agent

In [ ]:
from langchain.chat_models import init_chat_model

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages] # This means Datatype: "Sequence[BaseMessage]"; metadata = "add_message"

@tool
def add(a:int, b:int):
    """This is an addition function that adds 2 numbers"""
    return a + b

@tool
def subtract(a: int, b: int):
    """Subtraction function"""
    return a - b

@tool
def multiply(a: int, b: int):
    """Multiplication function"""
    return a * b

tools = [add, subtract, multiply]
model = init_chat_model("google_genai:gemini-2.0-flash").bind_tools(tools)

def model_call(state:AgentState) -> AgentState:
    system_prompt = SystemMessage(content="You are my AI assistant, answer my questions to the best of your ability.")
    response = model.invoke([system_prompt] + state["messages"])
    return {"messages": [response]}

def should_continue(state:AgentState) -> str:
    messages = state["messages"]
    last_message = messages[-1]
    
    # In LangChain and LangGraph, the ".tool_calls" attribute is found on AIMessage objects and represents a list of tool calls that a language model (LLM) has decided to make in response to a prompt. 
    if not last_message.tool_calls:
        return "end"
    else:
        return "continue"
    
graph = StateGraph(AgentState)
graph.add_node("our_agent", model_call)

tool_node = ToolNode(tools=tools)
graph.add_node("tools", tool_node)

graph.add_edge(START, "our_agent")
graph.add_conditional_edges(
    "our_agent",
    should_continue,
    {
        "end": END,
        "continue": "tools"
    },
)

graph.add_edge("tools", "our_agent")

app = graph.compile()

In [15]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

inputs = {"messages": [("user", "Add 40 + 12. Add 2 + 3")]}
print_stream(app.stream(inputs, stream_mode="values"))

================================ Human Message =================================

Add 40 + 12. Add 2 + 3
================================== Ai Message ==================================
Tool Calls:
  add (90e8247a-6316-4c57-86ec-527842d79435)
 Call ID: 90e8247a-6316-4c57-86ec-527842d79435
  Args:
    a: 40.0
    b: 12.0
  add (28f0ea11-2c29-490c-9d70-bbc6b4f653ed)
 Call ID: 28f0ea11-2c29-490c-9d70-bbc6b4f653ed
  Args:
    a: 2.0
    b: 3.0
================================= Tool Message =================================
Name: add

5
================================== Ai Message ==================================

The answer to 40 + 12 is 52 and the answer to 2 + 3 is 5.


In [16]:
inputs = {"messages": [("user", "Add 40 + 12 and then multiply the result by 6. Also tell me a joke please.")]}
print_stream(app.stream(inputs, stream_mode="values"))

================================ Human Message =================================

Add 40 + 12 and then multiply the result by 6. Also tell me a joke please.
================================== Ai Message ==================================
Tool Calls:
  add (1a3b2d3b-ecb3-450c-a7ad-888139c1f795)
 Call ID: 1a3b2d3b-ecb3-450c-a7ad-888139c1f795
  Args:
    a: 40.0
    b: 12.0
================================= Tool Message =================================
Name: add

52
================================== Ai Message ==================================
Tool Calls:
  multiply (033eb1a5-4160-482e-84c8-ed847c0daa4d)
 Call ID: 033eb1a5-4160-482e-84c8-ed847c0daa4d
  Args:
    a: 52.0
    b: 6.0
================================= Tool Message =================================
Name: multiply

312
================================== Ai Message ==================================

The answer is 312.

Why don't scientists trust atoms?

Because they make up everything!
